In [22]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [23]:
# Step 1: Load the Excel Sheet
file_path = ('Canada_Hosp1_COVID_InpatientData.xlsx')

# Read all sheets into separate DataFrames
admission_data = pd.read_excel(file_path, sheet_name='Data-at-admission')
days_breakdown_data = pd.read_excel(file_path, sheet_name='Days-breakdown')
hlos_data = pd.read_excel(file_path, sheet_name='Hospital-length-of-stay')
msl_data = pd.read_excel(file_path, sheet_name='Medication-Static-List')

In [24]:
#merge data
merged_data = admission_data.merge(hlos_data[['parent_id', 'hospital_length_of_stay']], left_on='id',right_on='parent_id', how='left')
merged_data.drop('parent_id', axis=1, inplace=True)
merged_data.head()

,id,reason_for_admission,age,sex,ethnicity,ethnicity_other,height,weight,comorbidities,comorbidities_other,...,pao2_fio2,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,hospital_length_of_stay
0,1,Fever [R50.9],74,Male,"""[]""",NaN,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""","GERD (gastroesophageal reflux disease), Dement...",...,NaN,NaN,NaN,NaN,1.1,NaN,NaN,223.0,NaN,21
1,2,Pneumonia [J18.9],61,Female,"""[]""",NaN,154.9,73.6,"""[\""Hypertension\"",\""Other\""]""","Closed fracture of fifth metatarsal bone, Dive...",...,NaN,NaN,NaN,NaN,1.0,NaN,2354,95.6,NaN,5
2,3,Pneumonia [J18.9],58,Female,"""[]""",NaN,NaN,NaN,"""[\""Hypertension\""]""",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
3,4,Suspected COVID-19 virus infection [U07.2],94,Male,"""[]""",NaN,182.8,66.2,"""[\""Hypertension\"",\""Other\""]""","Parkinson's Disease, Back pain, Benign Prostat...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
4,5,"Febrile respiratory illness [J98.9, R50.9]",91,Male,"""[]""",NaN,NaN,NaN,"""[\""Chronic cardiac disease (not hypertension)...","TAVI, Atrial fibrillation, Gastric Reflux, Pac...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [25]:
#filter data to keep only the given columns
columns_to_keep = ['id', 'age', 'sex', 'height', 'weight', 'comorbidities', 'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate', 'oxygen_saturation',
'temperature', 'motor', 'verbal', 'eye', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count',
'aptt_aptr', 'pt', 'alt', 'ast', 'serum_creatinine', 'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'pao2',
'pao2_fio2', 'ph', 'high_senstivity_cardiac_troponin', 'esr', 'inr', 'ferritin', 'd_dimer', 'crp', 'hs_crp', 'intubated', 'hospital_length_of_stay']
filtered_data = merged_data[columns_to_keep]
filtered_data.head()

,id,age,sex,height,weight,comorbidities,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,...,ph,high_senstivity_cardiac_troponin,esr,inr,ferritin,d_dimer,crp,hs_crp,intubated,hospital_length_of_stay
0,1,74,Male,NaN,NaN,"""[\""Hypertension\"",\""Diabetes\"",\""Other\""]""",99,62,85,20,...,NaN,NaN,NaN,1.1,NaN,NaN,223.0,NaN,No,21
1,2,61,Female,154.9,73.6,"""[\""Hypertension\"",\""Other\""]""",122,77,93,36,...,NaN,NaN,NaN,1.0,NaN,2354,95.6,NaN,No,5
2,3,58,Female,NaN,NaN,"""[\""Hypertension\""]""",109,72,73,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,7
3,4,94,Male,182.8,66.2,"""[\""Hypertension\"",\""Other\""]""",165,71,92,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,9
4,5,91,Male,NaN,NaN,"""[\""Chronic cardiac disease (not hypertension)...",131,76,60,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,9


In [26]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                508 non-null    int64  
 1   age                               508 non-null    int64  
 2   sex                               508 non-null    object 
 3   height                            272 non-null    float64
 4   weight                            358 non-null    float64
 5   comorbidities                     508 non-null    object 
 6   systolic_blood_pressure           508 non-null    int64  
 7   diastolic_blood_pressure          508 non-null    int64  
 8   heart_rate                        508 non-null    int64  
 9   respiratory_rate                  508 non-null    int64  
 10  oxygen_saturation                 508 non-null    int64  
 11  temperature                       508 non-null    float64
 12  motor   

In [27]:
# drop empty columns and columns with high number of null values
dropped_data = filtered_data

dropped_data = dropped_data.drop(columns=['high_senstivity_cardiac_troponin', 'hs_crp'])


threshold = 0.75

missing_percentage = dropped_data.isnull().mean()

columns_to_drop = missing_percentage[missing_percentage > threshold].index

print(f"Columns to drop due to high missing values: {list(columns_to_drop)}")

dropped_data = dropped_data.drop(columns=columns_to_drop)

print(f"Remaining columns after dropping: {dropped_data.columns}")



Columns to drop due to high missing values: ['pao2', 'pao2_fio2', 'ph', 'esr', 'ferritin']
Remaining columns after dropping: Index(['id', 'age', 'sex', 'height', 'weight', 'comorbidities',
       'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate',
       'respiratory_rate', 'oxygen_saturation', 'temperature', 'motor',
       'verbal', 'eye', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch',
       'mchc', 'rdw', 'platelet_count', 'aptt_aptr', 'pt', 'alt', 'ast',
       'serum_creatinine', 'sodium', 'potassium', 'total_serum_bilirubin',
       'lactate', 'inr', 'd_dimer', 'crp', 'intubated',
       'hospital_length_of_stay'],
      dtype='object')


In [28]:
# convert non int/float to dummies
converted_data = dropped_data
converted_data = pd.get_dummies(converted_data, columns=['sex', 'intubated', 'comorbidities'], 
                             drop_first=True, dtype=int)


In [29]:
converted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Columns: 173 entries, id to comorbidities_"[]"
dtypes: float64(25), int32(138), int64(8), object(2)
memory usage: 412.9+ KB


In [30]:
# convert no numerical into numerical
import numpy as np
def clean_numeric_column(column):
    return column.replace({'>': '', '<': ''}, regex=True).astype(float)

converted_data['aptt_aptr'] = clean_numeric_column(converted_data['aptt_aptr'])
converted_data['d_dimer'] = clean_numeric_column(converted_data['d_dimer'])


In [31]:
# impute missing values with mode/median
mode_col = ['motor', 'verbal', 'eye','aptt_aptr','sodium','potassium']
median_col = ['height', 'weight', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count',
'alt', 'ast','pt', 'serum_creatinine', 'total_serum_bilirubin', 'lactate', 'inr', 'd_dimer', 'crp']

In [32]:
cleaned_data = converted_data
from sklearn.impute import SimpleImputer
for col in median_col:
    cleaned_data[col] = cleaned_data[col].fillna(cleaned_data[col].median())
for col in mode_col:
    cleaned_data[col] = cleaned_data[col].fillna(cleaned_data[col].mode()[0])

In [43]:
cleaned_data2 = cleaned_data

In [44]:
# remove outliers
numeric_col = ['age', 'height', 'weight', 'systolic_blood_pressure', 'diastolic_blood_pressure', 'heart_rate', 'respiratory_rate', 'oxygen_saturation',
'temperature', 'motor', 'verbal', 'eye', 'wbc', 'rbc', 'hemoglobin', 'hematocrit', 'mcv', 'mch', 'mchc', 'rdw', 'platelet_count',
'aptt_aptr', 'pt', 'alt', 'ast', 'serum_creatinine', 'sodium', 'potassium', 'total_serum_bilirubin', 'lactate', 'inr', 'd_dimer', 'crp']

from scipy import stats
import numpy as np

def remove_outliers_zscore(df, columns, min_rows=450, z_threshold=3.0, step=0.1):
    current_threshold = z_threshold
    while df.shape[0] > min_rows:
        z_scores = np.abs(stats.zscore(df[columns]))
        df = df[(z_scores < current_threshold).all(axis=1)]
        
        if df.shape[0] < min_rows:
            current_threshold += step  # Relax the Z-score threshold to keep more rows
            df = df.copy()  # Restore original dataset if too many rows are removed
    
    return df

# Apply the function using Z-scores
df_filtered_z = remove_outliers_zscore(cleaned_data2, numeric_col, min_rows=450)

print(f"Original rows: {cleaned_data2.shape[0]}")
print(f"Filtered rows after Z-score: {df_filtered_z.shape[0]}")



Original rows: 508
Filtered rows after Z-score: 363


In [45]:
cleaned_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Columns: 173 entries, id to comorbidities_"[]"
dtypes: float64(27), int32(138), int64(8)
memory usage: 412.9 KB


In [46]:
#Statistics
 
# Number of rows and columns
print(f"Number of rows (patients): {df_filtered_z.shape[0]}")
print(f"Number of columns (features): {df_filtered_z.shape[1]}")

# Mean of each column
print("Mean of each column:")
print(df_filtered_z.mean())

# Check for columns with all blank values
all_blank_cols = df_filtered_z.columns[df_filtered_z.isna().all()]
print(f"Columns with all blank values: {all_blank_cols}")


Number of rows (patients): 363
Number of columns (features): 173
Mean of each column:
id                                                                          248.801653
age                                                                          65.388430
height                                                                      166.895592
weight                                                                       77.845455
systolic_blood_pressure                                                     129.157025
                                                                               ...    
comorbidities_"[\"Pregnancy\"]"                                               0.005510
comorbidities_"[\"Rheumatologic disorder\",\"Hematological\",\"Other\"]"      0.002755
comorbidities_"[\"Rheumatologic disorder\",\"Other\"]"                        0.002755
comorbidities_"[\"Stroke\",\"Other\"]"                                        0.005510
comorbidities_"[]"                          

In [47]:
# Normalize data
from sklearn.preprocessing import StandardScaler

# Separate features (X) and labels (y)
X = df_filtered_z.drop(columns=['hospital_length_of_stay'])
y = df_filtered_z['hospital_length_of_stay']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [48]:
# split data into training and testing
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [49]:
# feqature selection
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Store the original feature names before scaling
original_feature_names = X.columns 

# Initialize a Random Forest model
rf_model = RandomForestRegressor()

# Train the model
rf_model.fit(X_train, y_train)

# Get feature importance scores
importances = rf_model.feature_importances_

feature_importances = pd.DataFrame({
    'Feature': original_feature_names,  
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

selected_features = feature_importances['Feature'].head(10)


In [50]:
selected_features = selected_features[selected_features != 'id']
print("Top 10 Important Features:")
print(selected_features)

# Filter the dataset based on top features
X_train_selected = pd.DataFrame(X_train, columns=original_feature_names)[selected_features]
X_test_selected = pd.DataFrame(X_test, columns=original_feature_names)[selected_features]

Top 10 Important Features:
30                                              lactate
17                                                  mcv
21                                       platelet_count
82    comorbidities_"[\"Chronic cardiac disease (not...
2                                                height
16                                           hematocrit
5                              diastolic_blood_pressure
14                                                  rbc
3                                                weight
4                               systolic_blood_pressure
Name: Feature, dtype: object


In [51]:
# perform mlp
from sklearn.neural_network import MLPRegressor
# Initialize the MLP model with default parameters
mlp = MLPRegressor(max_iter=1000, random_state=42)

from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Scale the training data
X_train_selected_scaled = scaler.fit_transform(X_train_selected)

X_test_selected_scaled = scaler.transform(X_test_selected)

# Train the model using the scaled data
mlp.fit(X_train_selected_scaled, y_train)

# Predict and calculate MSE for the scaled data
y_train_pred = mlp.predict(X_train_selected_scaled)
y_test_pred = mlp.predict(X_test_selected_scaled)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Print the baseline MSE
print(f"Baseline Train MSE: {train_mse}")
print(f"Baseline Test MSE: {test_mse}")

Baseline Train MSE: 60.413374806257394
Baseline Test MSE: 142.06054401868096


c:\UB\Sem_2\Healthcare\Project1\hsolenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [52]:
# Hyperparametric tuning, model evaluation, cross validation
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Define the parameter grid with simpler networks
param_grid = {
    'hidden_layer_sizes': [(10,), (20,), (50,), (100,), (50, 50), (100, 50, 25)],  # Simple and more complex networks
    'activation': ['relu', 'tanh'],  # Different activation functions
    'alpha': [0.0001, 0.001, 0.01, 0.1],  # Regularization
    'solver': ['adam', 'sgd'],  # Different solvers
    'learning_rate': ['constant', 'adaptive'],  # Different learning rate schedules
}

# Initialize the MLPRegressor
mlp = MLPRegressor(max_iter=1000, random_state=42)

# Set up RepeatedKFold cross-validation with 5 splits and 5 trials
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Perform GridSearch with RepeatedKFold cross-validation
grid_search = GridSearchCV(mlp, param_grid, cv=cv, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the model with training data
grid_search.fit(X_train_selected_scaled, y_train)

# Best hyperparameters and estimator from grid search
best_params = grid_search.best_params_
best_mlp = grid_search.best_estimator_

# Cross-validation results
cv_results = grid_search.cv_results_

# Extract and display the relevant data
for mean_score, params in zip(cv_results['mean_test_score'], cv_results['params']):
    print(f"Mean MSE: {-mean_score:.4f} for params: {params}")

# Show the best cross-validation result
best_mean_mse = -grid_search.best_score_
print(f"Best CV MSE: {best_mean_mse}")

# Predict and calculate MSE on the test set
y_test_pred_tuned = best_mlp.predict(X_test_selected_scaled)
test_mse_tuned = mean_squared_error(y_test, y_test_pred_tuned)

# Final evaluation
print(f"Best Hyperparameters: {best_params}")
print(f"Tuned Test MSE: {test_mse_tuned}")


Fitting 25 folds for each of 192 candidates, totalling 4800 fits
Mean MSE: 139.2439 for params: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'solver': 'adam'}
Mean MSE: 149.7295 for params: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'solver': 'sgd'}
Mean MSE: 139.2439 for params: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive', 'solver': 'adam'}
Mean MSE: 152.5827 for params: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive', 'solver': 'sgd'}
Mean MSE: 138.1922 for params: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'solver': 'adam'}
Mean MSE: 144.5243 for params: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'constant', 'solver': 'sgd'}
Mean MSE: 138.1922 for params: {'activation': 'relu', 'a

c:\UB\Sem_2\Healthcare\Project1\hsolenv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
